In [3]:
def linclass(weight, data):
    data = np.insert(data,0,1,axis=1)
    class_pred = np.matmul(data,weight)
    class_pred[class_pred.flatten()>0,:] = 1
    class_pred[class_pred.flatten()<0,:] = -1
    return class_pred

In [5]:
import numpy as np


def leastSquares(data, label):
    # Sum of squared error shoud be minimized
    #
    # INPUT:
    # data        : Training inputs  (num_samples x dim)
    # label       : Training targets (num_samples x 1)
    #
    # OUTPUT:
    # weights     : weights   (dim x 1)
    # bias        : bias term (scalar)

    #####Start Subtask 1a#####
    # Extend each datapoint x as [1, x]
    # (Trick to avoid modeling the bias term explicitly)
    num_samples = len(data)
    data = np.concatenate((np.ones((num_samples, 1)), data), axis=1)  # before: (38x2), now: (38x3)

    # Take the pseudo inverse
    weight = np.linalg.lstsq((data.T.dot(data)),data.T)[0].dot(label) # inv(A)*b = A\b shape: (3, 1)  # Form the output

    bias = weight[0]  # get bias
    weight = weight[1:]  # get weights

    #####End Subtask#####
    return weight, bias


In [ ]:
def simpleClassifier(X, y, weights):
   
    
    classification_error = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        temp = np.zeros(X.shape[0])
        indices = np.argsort(X[:,i])
        sorted_X = np.sort(X[:,i])
        sorted_targets = y[indices]
        sorted_weights = weights[indices]
        for j in range(X.shape[0]):
            temp[j] = np.sum(sorted_weights[0:j+1] * sorted_targets[0:j+1])
        theta_max[i] = np.max(temp)
        theta_min[i] = np.min(temp)
        #print(temp)
    
        if(abs(theta_max[i])>abs(theta_min[i])):
            threshold_[i] = np.argmax(temp)
            sign[i] = 1
        else:
            threshold_[i] = np.argmin(temp)
            sign[i] = -1
        threshold[i] = sorted_X.flatten()[threshold_[i]]
       

    for i in range(threshold.shape[0]):
        hypo = np.zeros(X.shape[0])
        hypo[sign[i]*X[:,i] <= sign[i]*threshold[i]] = 1
        hypo[hypo==0] = -1
        indicator = hypo!=y.flatten()
        indicator = indicator.astype(int)
        #print((np.matmul(weights.transpose(),indicator)/np.sum(weights)).shape)
        classification_error[i] = np.matmul(weights.transpose(),indicator)/np.sum(weights)
    
    dimension = np.argmin(classification_error)
    parity_sign = sign[dimension]
    threshold_value = threshold[dimension]
    epsilon = classification_error[dimension]
    alpha = math.log((1-epsilon)/epsilon)
    hypothesis = np.zeros(X.shape[0])
    hypothesis[sign[dimension]*X[:,dimension] <= sign[dimension]*threshold[dimension]] = 1
    hypothesis[hypothesis==0] = -1
    indicator_ = hypothesis!=y.flatten()
    indicator_ = indicator_.astype(int)

    updated_weights = np.multiply(weights.flatten(),np.exp(np.multiply(alpha,indicator_)))
    return dimension, threshold_value, updated_weights, alpha